# 1.Import modules

In [ ]:
import matplotlib.pyplot as plt # For plotting
import numpy as np              # NumPy, for working with arrays/tensors 
import time                     # For measuring time
import random                   # Python's random library

import json
import glob
import os

from PIL import Image


# PyTorch libraries:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader


# 2.Enable GPU

In [ ]:
if torch.cuda.is_available():
  print("Cuda (GPU support) is available and enabled!")
  device = torch.device("cuda")
else:
  print("Cuda (GPU support) is not available :(")
  device = torch.device("cpu")

Cuda (GPU support) is available and enabled!


# 3.Download Placepedia

In [ ]:
!wget https://download.openmmlab.com/datasets/placepedia/Places-Fine.zip
!unzip Places-Fine.zip

--2021-07-27 23:20:46--  https://download.openmmlab.com/datasets/placepedia/Places-Fine.zip
Resolving download.openmmlab.com (download.openmmlab.com)... 47.88.36.78
Connecting to download.openmmlab.com (download.openmmlab.com)|47.88.36.78|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19223705488 (18G) [application/zip]
Saving to: ‘Places-Fine.zip’

Places-Fine.zip       0%[                    ]  44.09M  10.5MB/s    eta 34m 40s^C
Archive:  Places-Fine.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of Places-Fine.zip or
        Places-Fine.zip.zip, and cannot find Places-Fine.zip.ZIP, period.


# 4.Create custom pytorch dataset

In [ ]:
class PlacepediaDataset(Dataset):
    def __init__(self, train_json_path, image_folders_path, transform=None):
        super().__init__()
        self.transform = transform
        # initialize the arrays to store the ground truth labels and paths to the images
        self.data = []
        self.place_labels = []
        self.city_labels = []
        self.country_labels = []
        self.function_labels = []
        self.type_labels = []
     
        # read labels from the JSON file
        f = open(train_json_path)
        train_labels = json.load(f)
 
        print(json.dumps(train_labels,indent=4,sort_keys=False))
 
        for key in train_labels:
            folder_name = train_labels[key]['name_en']

            for img_path in os.listdir(image_folders_path + '/' + folder_name):
                self.data.append(image_folders_path + '/' + folder_name + '/' + img_path)
                self.place_labels.append(int(key))
                self.city_labels.append(train_labels[key]['city'])
                self.country_labels.append(train_labels[key]['country'])
                self.function_labels.append(train_labels[key]['function'])
                self.type_labels.append(train_labels[key]['type'])
 
    def __len__(self):
        return len(self.data)
 
    def __getitem__(self, idx):
        # take the data sample by its index
        img_path = self.data[idx]
    
        # read image
        img = Image.open(img_path)
    
        # apply the image augmentations if needed
        if self.transform:
            img = self.transform(img)
    
        # return the image and all the associated labels
        dict_data = {
            'img': img,
            'labels': {
                'place_labels': self.place_labels[idx],
                'city_labels': self.city_labels[idx],
                'country_labels': self.country_labels[idx],
                'function_labels': self.function_labels[idx],
                'type_labels': self.type_labels[idx]
            }
        }
        return dict_data

# 5.Construct PlaceNet

### 5.1 Duplicate last layers of ResNet50

In [ ]:
model_urls = {\
'resnet18': 'https://download.pytorch.org/models/resnet18-5c106cde.pth',\
'resnet34': 'https://download.pytorch.org/models/resnet34-333f7ec4.pth',\
'resnet50': 'https://download.pytorch.org/models/resnet50-19c8e357.pth',\
'resnet101': 'https://download.pytorch.org/models/resnet101-5d3b4d8f.pth',\
'resnet152': 'https://download.pytorch.org/models/resnet152-b121ed2d.pth',\
'resnext50_32x4d': 'https://download.pytorch.org/models/resnext50_32x4d-7cdf4587.pth',\
'resnext101_32x8d': 'https://download.pytorch.org/models/resnext101_32x8d-8ba56ff5.pth',\
'wide_resnet50_2': 'https://download.pytorch.org/models/wide_resnet50_2-95faca4d.pth',\
'wide_resnet101_2': 'https://download.pytorch.org/models/wide_resnet101_2-32ee1156.pth',\
}

def conv1x1(in_planes, out_planes, stride=1):
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)


def conv3x3(in_planes, out_planes, stride=1, groups=1, dilation=1):
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,padding=dilation, groups=groups, bias=False, dilation=dilation)


class BasicBlock(nn.Module):
    expansion = 1
 
    def __init__(self, inplanes, planes, stride=1, downsample=None, groups=1,
                 base_width=64, dilation=1, norm_layer=None):
        super(BasicBlock, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        if groups != 1 or base_width != 64:
            raise ValueError('BasicBlock only supports groups=1 and base_width=64')
        if dilation > 1:
            raise NotImplementedError("Dilation > 1 not supported in BasicBlock")
        # Both self.conv1 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = norm_layer(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = norm_layer(planes)
        self.downsample = downsample
        self.stride = stride
 
    def forward(self, x):
        identity = x
 
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
 
        out = self.conv2(out)
        out = self.bn2(out)
 
        if self.downsample is not None:
            identity = self.downsample(x)
 
        out += identity
        out = self.relu(out)
 
        return out


class Bottleneck(nn.Module):
    # Bottleneck in torchvision places the stride for downsampling at 3x3 convolution(self.conv2)
    # while original implementation places the stride at the first 1x1 convolution(self.conv1)
    # according to "Deep residual learning for image recognition"https://arxiv.org/abs/1512.03385.
    # This variant is also known as ResNet V1.5 and improves accuracy according to
    # https://ngc.nvidia.com/catalog/model-scripts/nvidia:resnet_50_v1_5_for_pytorch.
 
    expansion = 4
 
    def __init__(self, inplanes, planes, stride=1, downsample=None, groups=1,
                 base_width=64, dilation=1, norm_layer=None):
        super(Bottleneck, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        width = int(planes * (base_width / 64.)) * groups
        # Both self.conv2 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv1x1(inplanes, width)
        self.bn1 = norm_layer(width)
        self.conv2 = conv3x3(width, width, stride, groups, dilation)
        self.bn2 = norm_layer(width)
        self.conv3 = conv1x1(width, planes * self.expansion)
        self.bn3 = norm_layer(planes * self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride
 
    def forward(self, x):
        identity = x
 
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
 
        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)
 
        out = self.conv3(out)
        out = self.bn3(out)
 
        if self.downsample is not None:
            identity = self.downsample(x)
 
        out += identity
        out = self.relu(out)
 
        return out


class ResNet(nn.Module):
 
    def __init__(self, block, layers, num_classes=1000, 
                 zero_init_residual=False,
                 groups=1, width_per_group=64, replace_stride_with_dilation=None,
                 norm_layer=None):
        super(ResNet, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        self._norm_layer = norm_layer
 
        self.inplanes = 64
        self.dilation = 1
        if replace_stride_with_dilation is None:
            # each element in the tuple indicates if we should replace
            # the 2x2 stride with a dilated convolution instead
            replace_stride_with_dilation = [False, False, False]
        if len(replace_stride_with_dilation) != 3:
            raise ValueError("replace_stride_with_dilation should be None "
                             "or a 3-element tuple, got {}".format(replace_stride_with_dilation))
        self.groups = groups
        self.base_width = width_per_group
        self.conv1 = nn.Conv2d(3, self.inplanes, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = norm_layer(self.inplanes)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2,
                                       dilate=replace_stride_with_dilation[0])
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2,
                                       dilate=replace_stride_with_dilation[1])
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2,
                                       dilate=replace_stride_with_dilation[2])
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)
 
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
 
        # Zero-initialize the last BN in each residual branch,
        # so that the residual branch starts with zeros, and each residual block behaves like an identity.
        # This improves the model by 0.2~0.3% according to https://arxiv.org/abs/1706.02677
        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, Bottleneck):
                    nn.init.constant_(m.bn3.weight, 0)
                elif isinstance(m, BasicBlock):
                    nn.init.constant_(m.bn2.weight, 0)
 
    def _make_layer(self, block, planes, blocks, stride=1, dilate=False):
        norm_layer = self._norm_layer
        downsample = None
        previous_dilation = self.dilation
        if dilate:
            self.dilation *= stride
            stride = 1
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride),
                norm_layer(planes * block.expansion),
            )
 
        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample, self.groups,
                            self.base_width, previous_dilation, norm_layer))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes, groups=self.groups,
                                base_width=self.base_width, dilation=self.dilation,
                                norm_layer=norm_layer))
 
        return nn.Sequential(*layers)
 
    def _forward_impl(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
 
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
 
        x1 = self.layer4(x)
        x1 = self.avgpool(x1)
        x1 = torch.flatten(x1, 1)
        x1 = self.fc(x1)
 
        x2 = self.layer4(x)
        x2 = self.avgpool(x2)
        x2 = torch.flatten(x2, 1)
        x2 = self.fc(x2)
 
        x3 = self.layer4(x)
        x3 = self.avgpool(x3)
        x3 = torch.flatten(x3, 1)
        x3 = self.fc(x3)
 
        x4 = self.layer4(x)
        x4 = self.avgpool(x4)
        x4 = torch.flatten(x4, 1)
        x4 = self.fc(x4)
 
        x5 = self.layer4(x)
        x5 = self.avgpool(x5)
        x5 = torch.flatten(x5, 1)
        x5 = self.fc(x5)
        return x1,x2,x3,x4,x5
 
 
    def forward(self, x):
        return self._forward_impl(x)
        

def _resnet(arch, block, layers, pretrained, progress, **kwargs):
    model = ResNet(block, layers, **kwargs)
    if pretrained:
        state_dict = load_state_dict_from_url(model_urls[arch],
                                              progress=progress)
        model.load_state_dict(state_dict)
        
        
        
    return model

def PlaceNetBase(pretrained=False, progress=True, **kwargs):
    r"""ResNet-50 model from
    `"Deep Residual Learning for Image Recognition" <https://arxiv.org/pdf/1512.03385.pdf>`_
 
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _resnet('resnet50', Bottleneck, [3, 4, 6, 3], pretrained, progress,
                   **kwargs)

### 5.2 Add extra fc layers

In [ ]:
class PlaceNet(nn.Module):
    def __init__(self, pretrained, num_classes=1000, num_place = 444, num_city = 50, num_country = 34, num_function = 10, num_type = 50):
        super().__init__()
        self.base_model = PlaceNetBase(pretrained)  # take the model without classifier
        self.fc1 = nn.Linear(num_classes, num_place)
        self.fc2 = nn.Linear(num_classes, num_city)
        self.fc3 = nn.Linear(num_classes, num_country)
        self.fc4 = nn.Linear(num_classes, num_function)
        self.fc5 = nn.Linear(num_classes, num_type)
    
    def forward(self, x):
        x1,x2,x3,x4,x5 = self.base_model(x)
        x1 = self.fc1(x1)
        x2 = self.fc2(x2)
        x3 = self.fc3(x3)
        x4 = self.fc4(x4)
        x5 = self.fc5(x5)
        
        
        return {'place': x1,'city': x2,'country': x3,'function': x4,'type': x5}
    
    def get_loss(self, net_output, ground_truth):

        criterion = nn.CrossEntropyLoss()
        place_loss = criterion(net_output['place'], ground_truth['place_labels'])
        city_loss = criterion(net_output['city'], ground_truth['city_labels'])
        country_loss = criterion(net_output['country'], ground_truth['country_labels'])
        function_loss = criterion(net_output['function'], ground_truth['function_labels'])
        type_loss = criterion(net_output['type'], ground_truth['type_labels'])

        loss = place_loss + city_loss + country_loss + function_loss + type_loss
        return loss, {'place': place_loss, 'city': city_loss, 'country': country_loss, 'function': function_loss, 'type': type_loss}


# 6.Hyperparameters

In [ ]:
epochs = 90
# initial_lr = 0.05
initial_lr = 1e-4
lr_mult_rate = 0.1
weight_decay = 1e-4
momentum = 0.9
batch_size = 64 

num_workers = 0

# 7.Dataloader

In [ ]:
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[123,117,109], std=[58, 56, 58]),
])

test_val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[123,117,109], std=[58, 56, 58]),
])

train_dataset = PlacepediaDataset('Places-Fine/train_target_dict.json', 'Places-Fine/train', train_transform) # TODO: bu pathleri düzelt
train_data_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)


FileNotFoundError: ignored

# 8.Accuracy function

In [ ]:
def calcTopKaccuracy(k,output,target):
    batch_size = target['place_labels'].cpu().size(0)
    _ , place_indices = torch.topk(output['place'].cpu(), k)
    _ , city_indices = torch.topk(output['city'].cpu(), k)
    _ , country_indices = torch.topk(output['country'].cpu(), k)
    _ , function_indices = torch.topk(output['function'].cpu(), k)
    _ , type_indices = torch.topk(output['type'].cpu(), k)
    
    place_count = 0
    city_count = 0
    country_count = 0
    function_count = 0
    type_count = 0
    
    for i in range(batch_size):
        if target['place_labels'][i].cpu().item() in place_indices[i]:
            place_count += 1
        if target['city_labels'][i].cpu().item() in city_indices[i]:
            city_count += 1
        if target['country_labels'][i].cpu().item() in country_indices[i]:
            country_count += 1
        if target['function_labels'][i].cpu().item() in function_indices[i]:
            function_count += 1
        if target['type_labels'][i].cpu().item() in type_indices[i]:
            type_count += 1
    
    place_acc = place_count/batch_size*100
    city_acc = city_count/batch_size*100
    country_acc = country_count/batch_size*100
    function_acc = function_count/batch_size*100
    type_acc = type_count/batch_size*100
    
    return {'place':place_acc,'city':city_acc,'country':country_acc,'function':function_acc,'type':type_acc}


# 9.Trainer

In [ ]:
def train(model, optimizer, scheduler, epochs, train_data_loader, batch_size, verbose=True):

    total_loss_history = [] 
    loss_history_place = []
    loss_history_city = []
    loss_history_country = []
    loss_history_function = []
    loss_history_type = []

    total_top5_place_acc = 0
    total_top5_city_acc = 0
    total_top5_country_acc = 0
    total_top5_function_acc = 0
    total_top5_type_acc = 0
    
    total_top1_place_acc = 0
    total_top1_city_acc = 0
    total_top1_country_acc = 0
    total_top1_function_acc = 0
    total_top1_type_acc = 0
    
    for epoch in range(epochs):
        top5_place_acc = 0
        top5_city_acc = 0
        top5_country_acc = 0
        top5_function_acc = 0
        top5_type_acc = 0
        
        top1_place_acc = 0
        top1_city_acc = 0
        top1_country_acc = 0
        top1_function_acc = 0
        top1_type_acc = 0
        
        for i, batch in enumerate(train_data_loader,0):    
            # Our batch:
            img = batch['img']
            target_labels = batch['labels']
            target_labels = {t: target_labels[t].to(device) for t in target_labels}
            
            optimizer.zero_grad()

            outputs = model(img.to(device))
         
            outputs = {t: outputs[t].to(device) for t in outputs}
            
            # Calculate loss
            loss, losses = model.get_loss(outputs, target_labels)
            
            # Calculate top-k accuracy
            top5_accs = calcTopKaccuracy(5,outputs,target_labels)
            top1_accs = calcTopKaccuracy(1,outputs,target_labels)
            
            top5_place_acc = top5_place_acc + top5_accs['place']
            top5_city_acc = top5_city_acc + top5_accs['city']
            top5_country_acc = top5_country_acc + top5_accs['country']
            top5_function_acc = top5_function_acc + top5_accs['function']
            top5_type_acc = top5_type_acc + top5_accs['type']
            
            top1_place_acc = top1_place_acc + top1_accs['place']
            top1_city_acc = top1_city_acc + top1_accs['city']
            top1_country_acc = top1_country_acc + top1_accs['country']
            top1_function_acc = top1_function_acc + top1_accs['function']
            top1_type_acc = top1_type_acc + top1_accs['type']
            
            # save loss history
            loss_history_place.append(losses['place'].item())
            loss_history_city.append(losses['city'].item())
            loss_history_country.append(losses['country'].item())
            loss_history_function.append(losses['function'].item())
            loss_history_type.append(losses['type'].item())
            
            # Backpropagate
            loss.backward()
            
            # Update the weights
            optimizer.step()
            
            # Update learning rate
            scheduler.step()
            
            total_loss_history.append(loss.item())
            
        total_top5_place_acc = total_top5_place_acc + top5_place_acc
        total_top5_city_acc = total_top5_city_acc + top5_city_acc
        total_top5_country_acc = total_top5_country_acc + top5_country_acc
        total_top5_function_acc = total_top5_function_acc + top5_function_acc
        total_top5_type_acc = total_top5_type_acc + top5_type_acc
        
        total_top1_place_acc = total_top1_place_acc + top1_place_acc
        total_top1_city_acc = total_top1_city_acc + top1_city_acc
        total_top1_country_acc = total_top1_country_acc + top1_country_acc
        total_top1_function_acc = total_top1_function_acc + top1_function_acc
        total_top1_type_acc = total_top1_type_acc + top1_type_acc
        
        
        if verbose: print(f'Epoch {epoch} / {epochs}: avg. loss of last 5 iterations {np.sum(total_loss_history[:-6:-1])/5}')
        
        if verbose: print('Epoch', epoch ,' /', epochs, '   loss :', loss)
        if verbose: print('Epoch', epoch ,' /', epochs, '   top5 place accuracy : %', top5_place_acc/(i+1))
        if verbose: print('Epoch', epoch ,' /', epochs, '   top5 city accuracy : %', top5_city_acc/(i+1))
        if verbose: print('Epoch', epoch ,' /', epochs, '   top5 country accuracy : %', top5_country_acc/(i+1))
        if verbose: print('Epoch', epoch ,' /', epochs, '   top5 function accuracy : %', top5_function_acc/(i+1))
        if verbose: print('Epoch', epoch ,' /', epochs, '   top5 type accuracy : %', top5_type_acc/(i+1))
        if verbose: print('Epoch', epoch ,' /', epochs, '   top1 place accuracy : %', top1_place_acc/(i+1))
        if verbose: print('Epoch', epoch ,' /', epochs, '   top1 city accuracy : %', top1_city_acc/(i+1))
        if verbose: print('Epoch', epoch ,' /', epochs, '   top1 country accuracy : %', top1_country_acc/(i+1))
        if verbose: print('Epoch', epoch ,' /', epochs, '   top1 function accuracy : %', top1_function_acc/(i+1))
        if verbose: print('Epoch', epoch ,' /', epochs, '   top1 type accuracy : %', top1_type_acc/(i+1))
    
    top5_acc = {'place':total_top5_place_acc,'city':total_top5_city_acc,'country':total_top5_country_acc,'function':total_top5_function_acc,'type':total_top5_type_acc}
    top1_acc = {'place':total_top1_place_acc,'city':total_top1_city_acc,'country':total_top1_country_acc,'function':total_top1_function_acc,'type':total_top1_type_acc}
    loss_history = {'place':loss_history_place,'city':loss_history_city,'country':loss_history_country,'function':loss_history_function,'type':loss_history_type}
    
    return total_loss_history, loss_history, top5_acc, top1_acc


# 10.Define and create model, optimizer, scheduler 

In [ ]:
model = PlaceNet(pretrained=True).to(device)
optimizer = optim.SGD(model.parameters(), lr=initial_lr, momentum=momentum, weight_decay=weight_decay) #https://pytorch.org/docs/stable/generated/torch.optim.SGD.html
scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[63,81], gamma=lr_mult_rate) #https://pytorch.org/docs/stable/generated/torch.optim.lr_scheduler.MultiStepLR.html#torch.optim.lr_scheduler.MultiStepLR


# 11.Train

In [ ]:
total_loss_history, loss_history, top5_acc, top1_acc = train(model, optimizer, scheduler, num_epochs, train_data_loader, batch_size)

# 12.Total Loss Curve

In [ ]:
plt.plot(total_loss_history)
plt.xlabel('Iteration number')
plt.ylabel('Loss value')
plt.show()

# 13.More results (accuracy, loss per output)

In [ ]:
total_top5_place_acc = top5_acc['place'] 
total_top5_city_acc = top5_acc['city']
total_top5_country_acc = top5_acc['country']
total_top5_function_acc = top5_acc['function']
total_top5_type_acc = top5_acc['type']

total_top1_place_acc = top1_acc['place'] 
total_top1_city_acc = top1_acc['city']
total_top1_country_acc = top1_acc['country']
total_top1_function_acc = top1_acc['function']
total_top1_type_acc = top1_acc['type']

loss_history_place = loss_history['place']
loss_history_city = loss_history['city']
loss_history_country = loss_history['country']
loss_history_function = loss_history['function']
loss_history_type = loss_history['type']
    
plt.plot(loss_history_place)
plt.xlabel('Iteration number')
plt.ylabel('Place loss value')
plt.show()

plt.plot(loss_history_city)
plt.xlabel('Iteration number')
plt.ylabel('City loss value')
plt.show()

plt.plot(loss_history_country)
plt.xlabel('Iteration number')
plt.ylabel('Country loss value')
plt.show()

plt.plot(loss_history_function)
plt.xlabel('Iteration number')
plt.ylabel('Function loss value')
plt.show()

plt.plot(loss_history_type)
plt.xlabel('Iteration number')
plt.ylabel('Type loss value')
plt.show()